In [21]:
# 1、load
from langchain.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
data = loader.load()

In [22]:
# 2、Split
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 0)
all_splits = text_splitter.split_documents(data)

In [23]:
# 3、Store
from langchain.vectorstores import Chroma
from langchain.embeddings import GPT4AllEmbeddings

vectorstore = Chroma.from_documents(documents=all_splits, embedding=GPT4AllEmbeddings())

Found model file at  /home/zk/.cache/gpt4all/ggml-all-MiniLM-L6-v2-f16.bin


In [24]:
# 4、Retrieve
question = "What are the approaches to Task Decomposition?"
docs = vectorstore.similarity_search(question)

In [25]:
# 5、Generate

# 实例化一个大模型
from langchain.llms import GPT4All
llm = GPT4All(model="../models/ggml-gpt4all-j-v1.3-groovy.bin", max_tokens=2048, verbose=True, allow_download=True)

from langchain.chains import RetrievalQA
qa_chain = RetrievalQA.from_chain_type(llm,retriever=vectorstore.as_retriever())
qa_chain({"query": question})

Found model file at  ./models/ggml-gpt4all-j-v1.3-groovy.bin
gptj_model_load: loading model from './models/ggml-gpt4all-j-v1.3-groovy.bin' - please wait ...
gptj_model_load: n_vocab = 50400
gptj_model_load: n_ctx   = 2048
gptj_model_load: n_embd  = 4096
gptj_model_load: n_head  = 16
gptj_model_load: n_layer = 28
gptj_model_load: n_rot   = 64
gptj_model_load: f16     = 2
gptj_model_load: ggml ctx size = 5401.45 MB
gptj_model_load: kv self size  =  896.00 MB
gptj_model_load: ................................... done
gptj_model_load: model size =  3609.38 MB / num tensors = 285


{'query': 'What are the approaches to Task Decomposition?',
 'result': ' The following are some of the common methods used for task decomposition in long-term planning:\n1. LLM with simple prompting like "Steps for XYZ.\\n1.", "What are the subgoaalls for achieving XYZ?"\n2. Using task-specific instructions; e.g. "Write a story outline." for writing a novel, or (3) with human inputs.'}

In [26]:
# 6、Converse (Extension)
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

from langchain.chains import ConversationalRetrievalChain
chat = ConversationalRetrievalChain.from_llm(llm, retriever=vectorstore.as_retriever(), memory=memory)

In [27]:
result = chat({"question": "What are the approaches to Task Decomposition?"})
result['answer']

' The following are some of the common methods used for task decomposition in long-term planning:\n1. LLM with simple prompting like "Steps for XYZ.\\n1.", "What are the subgoaalls for achieving XYZ?"\n2. Using task-specific instructions; e.g. "Write a story outline." for writing a novel, or (3) with human inputs.'

In [28]:
result = chat({"question": "How does the Reflexion paper handle it?"})
result['answer']

' The context provided does not contain information about different approaches or comparisons between them, so I am unable to answer this question.'